In [2]:
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import sys
from PIL import Image, ImageDraw, ImageFont
import json
import cv2
import os
import time
import tensorflow as tf
from keras import layers, Input, models, optimizers
from keras.models import Model, Sequential
from keras.models import load_model
from keras import layers, Input, models, optimizers
from keras.layers import MaxPooling2D, Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

In [3]:
model_pattern = './model/pattern_9985.h5'
var_pattern =  ['Check', 'Dot', 'Floral', 'Graphic', 'Leopard', 'None', 'Stripe']

model_top = './model/upper_9978.h5'
var_top = ['Hoodie', 'Long-sleeve-tee', 'Shirts', 'Short-sleeve-tee', 'Sweater',]

model_bottom = './model/lower0425_9990.h5'
var_bottom = ['Chiffon-skirt', 'Cotten-long-pants', 'Cutoffs', 'H-line-skirt', 'Jeans']

model_style = './model/style_model_1_90,63.h5'
var_style =  ['Casual', 'Feminine', 'Formal']

In [4]:
conv_base = VGG16(weights = 'imagenet',include_top=False,input_shape=(150,150,3))

top_model = models.Sequential()
top_model.add(layers.Dense(256, activation='relu', input_dim = 4 * 4 * 512))
top_model.add(layers.Dropout(0.5))
top_model.add(layers.Dense(4, activation='softmax'))
top_model.compile(optimizer = optimizers.RMSprop(lr=2e-5),
                      loss='categorical_crossentropy',
                      metrics=['acc'])
top_model.load_weights(model_top)

In [5]:
pattern_model = models.Sequential()
pattern_model.add(layers.Dense(256, activation='relu', input_dim = 4 * 4 * 512))
pattern_model.add(layers.Dropout(0.5))
pattern_model.add(layers.Dense(7, activation='softmax'))
pattern_model.compile(optimizer = optimizers.RMSprop(lr=2e-5),
                      loss='categorical_crossentropy',
                      metrics=['acc'])
pattern_model.load_weights(model_pattern)

In [6]:
model_bottom = load_model(model_bottom)
model_style = load_model(model_style)

In [10]:
img_dir = './data/1.jpg'
img = Image.open(img_dir)
img = img.resize((150,150))
data = np.array(img)
data = data.astype('float')
data = data/255
data = data.reshape((1,150,150,3))

상의 라면

In [ ]:
p = conv_base.predict(data)
p = p.reshape((-1,8192))
p = pattern_model.predict_classes(p)[0]
tp = var_pattern[p]

s = conv_base.predict(data)
s = s.reshape((-1,8192))
s = top_model.predict_classes(s)[0]
ts = var_top[s]

하의 라면

In [ ]:
p = conv_base.predict(data)
p = p.reshape((-1,8192))
p = pattern_model.predict_classes(p)[0]
bp = var_pattern[p]           

s = conv_base.predict(data)
s = s.reshape((-1,8192))
s = model_bottom.predict_classes(s)[0]
bs = var_bottom[s]

In [14]:
def identify_clothes(data,up_lo): # 사진 데이터과 상의 이지 하의 인지 상의 :0
    if up_lo==0: # 상의 라면
        p = conv_base.predict(data)
        p = p.reshape((-1,8192))
        p = pattern_model.predict_classes(p)[0]
        tp = var_pattern[p]
        
        s = conv_base.predict(data)
        s = s.reshape((-1,8192))
        s = top_model.predict_classes(s)[0]
        ts = var_top[s]
        
        return tp, ts
    else: # 하의 라면
        p = conv_base.predict(data)
        p = p.reshape((-1,8192))
        p = pattern_model.predict_classes(p)[0]
        bp = var_pattern[p]           
        
        s = conv_base.predict(data)
        s = s.reshape((-1,8192))
        s = model_bottom.predict_classes(s)[0]
        bs = var_bottom[s]
        
        return bp, bs

        

In [24]:
def recommend(df, style, pattern, shape): # dataframe  스타일, identify_clothes(data,0)[0], identify_clothes(data,0)[1]
    if style == 'Casual':
        ca_df = df[df['style']=='Casual']
        ca_df = ca_df[['top shape','top pattern', 'bottom shape', 'bottom pattern']]
        ca_df = ca_df.dropna(axis = 0)
        ca_df['zeros'] = 1
        ca_table=ca_df.pivot_table('zeros',index=['top shape','top pattern'],columns=['bottom shape', 'bottom pattern'], aggfunc='count')
        table = ca_table.fillna(0)
        
    elif style =="Feminine":
        fe_df = df[df['style']=='Feminine']
        fe_df = fe_df[['top shape','top pattern', 'bottom shape', 'bottom pattern']]
        fe_df = fe_df.dropna(axis = 0)
        fe_df['zeros'] = 1
        fe_table= fe_df.pivot_table('zeros',index=['top shape','top pattern'],columns=['bottom shape', 'bottom pattern'], aggfunc='count')
        table = fe_table.fillna(0)
        
    else:
        fo_df = df[df['style']=='Formal']
        fo_df = fo_df[['top shape','top pattern', 'bottom shape', 'bottom pattern']]
        fo_df = fo_df.dropna(axis = 0)
        fo_df['zeros'] = 1
        fo_table= fo_df.pivot_table('zeros',index=['top shape','top pattern'],columns=['bottom shape', 'bottom pattern'], aggfunc='count')
        table = fo_table.fillna(0)
        
        
    if shape in ['Hoodie', 'Long-sleeve-tee', 'Shirts', 'Short-sleeve-tee', 'Sweater']:
        matched_lower = table.loc[(shape, pattern)].sort_values(ascending=False).index[0]
        matched_shape = matched_lower[0]
        matched_pattern = matched_lower[1]
    else :
        matched_upper = f_df2.xs((shape,pattern), axis=1).sort_values(ascending=False).index[0]
        matched_shape = matched_upper[0]
        matched_pattern = matched_upper[1]
    return matched_shape, matched_pattern
            
            

In [9]:
df = pd.read_csv('./data/matrix_codibook_1000.csv')

In [19]:
img_dir = './data/8.jpg'
img = Image.open(img_dir)
img = img.resize((150,150))
data = np.array(img)
data = data.astype('float')
data = data/255
data = data.reshape((1,150,150,3))

In [34]:
identify_clothes(data,0)

('None', 'Long-sleeve-tee')

In [33]:
recommend(df, "Casual", identify_clothes(data,0)[0], identify_clothes(data,0)[1])

('Cotten-long-pants', 'None')